## Create folds
Notebook to create K folds for data train and validation and save it in dataframe format

It will get the quantity of classes automatically

In [46]:
'''
CSV_INPUT_ALL_CLASSES: Path to csv file containing all the dataset
OUTPUT_TRAIN_FOLDER: Folder to save split TRAIN folds
OUTPUT_TEST_FOLDER: Folder to save split TEST folds
K_FOLDS: Folds number to be generated
'''
DATA_FOLDER='finalv1/'
# CSV_INPUT_ALL_CLASSES='../csv/input/all_classes_hmv_hcpa.csv'
CSV_INPUT_ALL_CLASSES='../csv/input/all_classes_hmv_hcpa_SAMPLE.csv'
OUTPUT_TRAIN_FOLDER='../csv/input/'+ DATA_FOLDER +'train/'
OUTPUT_TEST_FOLDER='../csv/input/'+ DATA_FOLDER +'test/'
K_FOLDS=4

In [47]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import StratifiedKFold

import glob
import os
from datetime import datetime

In [48]:
'''
Make all paths exist
'''
if not os.path.exists('../csv/'): 
    os.mkdir('../csv/')
if not os.path.exists('../csv/input/'): 
    os.mkdir('../csv/input/')
if not os.path.exists('../csv/input/'+DATA_FOLDER): 
    os.mkdir('../csv/input/'+DATA_FOLDER)
if not os.path.exists(OUTPUT_TRAIN_FOLDER): 
    os.mkdir(OUTPUT_TRAIN_FOLDER)
if not os.path.exists(OUTPUT_TEST_FOLDER): 
    os.mkdir(OUTPUT_TEST_FOLDER)

In [49]:
df = pd.read_csv(CSV_INPUT_ALL_CLASSES)
df

,nome,covid
0,NEG-002,NEGATIVE
1,NEG-004,NEGATIVE
2,NEG-007,NEGATIVE
3,NEG-009,NEGATIVE
4,NEG-012,NEGATIVE
...,...,...
235,N241,ATYPICAL
236,N248,ATYPICAL
237,N270,ATYPICAL
238,N271,ATYPICAL


In [50]:
# Get all classes in the dataframe
ar_classes = df.covid.unique()
ar_classes.sort()
ar_classes

array(['ATYPICAL', 'INDETERMINATE', 'NEGATIVE', 'POSITIVE'], dtype=object)

In [51]:
df['int_covid'] = 0
for i in range(len(ar_classes)):
    df.loc[ df.covid==ar_classes[i], 'int_covid'] = i
df

# print(len(df[df['covid'] == 'covid']))
# print(len(df[df['covid'] == 'non covid']))

,nome,covid,int_covid
0,NEG-002,NEGATIVE,2
1,NEG-004,NEGATIVE,2
2,NEG-007,NEGATIVE,2
3,NEG-009,NEGATIVE,2
4,NEG-012,NEGATIVE,2
...,...,...,...
235,N241,ATYPICAL,0
236,N248,ATYPICAL,0
237,N270,ATYPICAL,0
238,N271,ATYPICAL,0


Generate the split to train and test samples.

In [52]:
X = df.index.to_list()
y = df['int_covid'].to_list()
skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True)
skf.get_n_splits(X, y)
# print(skf)
j = 1
for train_index, test_index in skf.split(X, y):
    df_train = pd.DataFrame(columns=['nome', 'covid'])
    for i in train_index:
        df_train = df_train.append(df.iloc[i])
    df_train = df_train.drop(columns=['int_covid'])
    df_train.to_csv( OUTPUT_TRAIN_FOLDER + 'train'+str(j)+'.csv', index=False)
    df_train.loc[df_train['covid'] == 'ATYPICAL', 'covid'] = 'others'
    df_train.loc[df_train['covid'] == 'INDETERMINATE', 'covid'] = 'others'
    df_train.loc[df_train['covid'] == 'NEGATIVE', 'covid'] = 'others'
    df_train.loc[df_train['covid'] == 'POSITIVE', 'covid'] = 'covid'
    df_train.to_csv(OUTPUT_TRAIN_FOLDER+'train'+str(j)+'_binario.csv', index=False)
    
    df_test = pd.DataFrame(columns=['nome', 'covid'])
    for i in test_index:
        df_test = df_test.append(df.iloc[i])
    df_test = df_test.drop(columns=['int_covid'])
    df_test.to_csv(OUTPUT_TEST_FOLDER+'test'+str(j)+'.csv', index=False)
    df_test.loc[df_test['covid'] == 'ATYPICAL', 'covid'] = 'others'
    df_test.loc[df_test['covid'] == 'INDETERMINATE', 'covid'] = 'others'
    df_test.loc[df_test['covid'] == 'NEGATIVE', 'covid'] = 'others'
    df_test.loc[df_test['covid'] == 'POSITIVE', 'covid'] = 'covid'
    df_test.to_csv(OUTPUT_TEST_FOLDER+'test'+str(j)+'_binario.csv', index=False)
    j+=1


# Test for 1 fold

In [45]:
s_a = df[df['covid'] == "ATYPICAL"].sample(40).index
s_i = df[df['covid'] == "INDETERMINATE"].sample(40).index
s_n = df[df['covid'] == "NEGATIVE"].sample(40).index
s_p = df[df['covid'] == "POSITIVE"].sample(120).index


df2 = df.loc[s_a.union(s_i).union(s_n).union(s_p)]
df2.index
print(len(df2))
print(df2.index)
df2.to_csv('../csv/input/all_classes_hmv_hcpa_SAMPLE.csv', index=False)

df_test = df[ ~df.isin(df2)].dropna()

print(len(df_test))
print(df_test.index)
df_test.to_csv('../csv/input/all_classes_hmv_hcpa_TEST.csv', index=False)

240
Int64Index([  1,   3,   6,   8,  11,  15,  16,  17,  18,  19,
            ...
            366, 368, 371, 372, 374, 376, 378, 380, 381, 382],
           dtype='int64', length=240)
143
Int64Index([  0,   2,   4,   5,   7,   9,  10,  12,  13,  14,
            ...
            359, 362, 363, 367, 369, 370, 373, 375, 377, 379],
           dtype='int64', length=143)
